# Task 3
\
In this task we create GDP growth figures for every country in separate charts and save them as a PNG file


First we read the data from the .pkl. Note: Please run the Setup script if not done yet.

In [1]:
import pandas as pd
imf_data = pd.read_pickle("IMF_DATA.pkl")

Then we filter the data we want to keep.
\
In this step we must be careful of the amount of data we leave as there are more then one Subject Descriptor that contains the gdp_pattern, however only 'Gross domestic product, current prices' is denominated in U.S. dollars and has 195 rows which fits are all the countries we need. 
\
**Note: GDP figures are scaled to billions of dollars**


In [2]:
gdp_pattern = 'Gross domestic product.(?!per capita).*'
gdp_data = imf_data[imf_data['Subject Descriptor'].str.contains(gdp_pattern, regex = True)]

usd_pattern = 'U.S. dollars'
gdp_data = gdp_data[gdp_data['Units'].str.contains(usd_pattern, regex = True)]

In [3]:
# This function sets the predictions to 0
def remove_predictions(row):
    DATA_END_DATE = 2025
    DATA_START_DATE = 1980
    
    prediction_start_year = int(row['Estimates Start After'])
    
    if prediction_start_year < DATA_START_DATE:
        row.loc[DATA_START_DATE : DATA_END_DATE] = 0
    elif prediction_start_year < DATA_END_DATE:
        row.loc[prediction_start_year + 1 : DATA_END_DATE] = 0
        
    return row

# Applying the predictions removal function and filtering the data, we keep the 0 values for now
gdp_data.set_index('Country', inplace = True)
gdp_data = gdp_data.apply(remove_predictions, axis = 1)
gdp_data = gdp_data.loc[:, 1980 : 2025]
gdp_data = gdp_data.reset_index()

Next we create a folder for the graphs if one does not exist

In [4]:
import os
FOLDER_NAME = 'IMF 2020 OCT GDP graphs'
folderExists = os.path.exists(FOLDER_NAME)

if not folderExists:
    os.makedirs(FOLDER_NAME)

I had some problem not plotting 0 values so i will just melt the gdp_data, remove the zero values and then pivot it, thus making nan values which matplotlib does not plot.

In [5]:
gdp_data = gdp_data.melt(id_vars = 'Country', var_name='Year', value_name='GDP')
row_indexes_to_be_dropped = gdp_data[gdp_data['GDP'] == 0].index

gdp_data = gdp_data.drop(row_indexes_to_be_dropped)
gdp_data = gdp_data.pivot(index='Country', columns='Year', values='GDP')

/var/folders/80/d8cfy0f16hl333cl0t5pf9540000gn/T/ipykernel_55899/1396145529.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  gdp_data = gdp_data.pivot(index='Country', columns='Year', values='GDP')


Now we loop over the rows of the gdp data and firstly make a long dataframe out of the country and then making a linechart which is saved to the folder specified above

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

# Looping over every country and saving the graph into FOLDER_NAME
for country, gdp in gdp_data.groupby(gdp_data.index):
    plt.figure()  # Create a new figure for each country
    sns.lineplot(data=gdp.T)  # Transpose the data for plotting
    plt.title(f"GDP for {country} in billions")
    plt.xlabel('Year')
    plt.ylabel('GDP in billions of U.S. dollars')
    plt.legend(loc='best')
    plt.grid(True)
    graph_name = '{}/{}_graph.png'.format(FOLDER_NAME, country)
    plt.savefig(graph_name)
    plt.close()

print(f"Graphs uploaded to {FOLDER_NAME}")

Graphs uploaded to IMF 2020 OCT GDP graphs


Now the graphs are in the specified foler\
**Note: the graphs only show actual data and no predictions**